In [151]:
import pandas as pd

symbol = "BNBEUR"

df = pd.read_csv(f"../binance-crawler/data/{symbol}/20201231_2259-20210401_2259.csv")
df = df.drop(columns=['ignore', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'quote_asset_volume', 'num_trades' ])
df['open_time'] = pd\
    .to_datetime(df['open_time'], unit='ms')\
    .dt.tz_localize('UTC')\
    .dt.tz_convert('Europe/Amsterdam')

df['close_time'] = pd\
    .to_datetime(df['close_time'], unit='ms')\
    .dt.tz_localize('UTC')\
    .dt.tz_convert('Europe/Amsterdam')

df.set_index('open_time', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6290 entries, 2021-01-01 01:00:00+01:00 to 2021-03-07 16:00:00+01:00
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype                           
---  ------      --------------  -----                           
 0   open        6290 non-null   float64                         
 1   high        6290 non-null   float64                         
 2   low         6290 non-null   float64                         
 3   close       6290 non-null   float64                         
 4   volume      6290 non-null   float64                         
 5   close_time  6290 non-null   datetime64[ns, Europe/Amsterdam]
dtypes: datetime64[ns, Europe/Amsterdam](1), float64(5)
memory usage: 344.0 KB


In [152]:
import btalib

df['MA_3'] = btalib.sma(df.close, period=9).df
df['RSI'] = btalib.rsi(df.close).df
df['EMA_3'] = btalib.ema(df.close, period=3).df

bta_macd = btalib.macd(df.close).df
df['MACD'] = bta_macd['macd']
df['Signal'] = bta_macd['signal']
df['Histogram'] = bta_macd['histogram']

feb = pd.to_datetime("2021-02-01")\
    .tz_localize('Europe/Amsterdam')
month = df[df.index >= feb]

month.tail()

,open,high,low,close,volume,close_time,MA_3,RSI,EMA_3,MACD,Signal,Histogram
open_time,,,,,,,,,,,,
2021-03-07 15:00:00+01:00,199.7718,201.5538,199.4456,200.2056,2192.684,2021-03-07 15:14:59.999000+01:00,200.171989,58.163716,200.009951,1.560295,2.027403,-0.467109
2021-03-07 15:15:00+01:00,200.2431,200.5154,197.1027,198.5090,2213.743,2021-03-07 15:29:59.999000+01:00,199.836256,50.670728,199.259475,1.324760,1.886875,-0.562115
2021-03-07 15:30:00+01:00,198.4323,198.8120,197.9000,198.5000,780.513,2021-03-07 15:44:59.999000+01:00,199.771333,50.633464,198.879738,1.124409,1.734381,-0.609972
2021-03-07 15:45:00+01:00,198.4172,199.3704,198.4172,199.1984,1697.068,2021-03-07 15:59:59.999000+01:00,199.600711,53.491762,199.039069,1.010338,1.589573,-0.579235
2021-03-07 16:00:00+01:00,199.1984,199.1984,198.4500,199.1729,229.362,2021-03-07 16:14:59.999000+01:00,199.419933,53.370257,199.105984,0.907418,1.453142,-0.545724


In [153]:
coins_bought = [1, 1, 1]
margin = [0, 0, 0]
times_bought = [0, 0, 0]
buy = [0.0, 0.0, 0.0]
is_bought = [False, False, False]
buy_time = None
for i in range(len(month)):
    if i % 672 == 0:
        print(f"""
        Week {i/672}
        Margin : {margin},
        Times Bought : {times_bought},
        """)

    candle = month.iloc[i]
    # 0
    if candle['Histogram'] > candle['Signal']:
        if not is_bought[0]:
            times_bought[0] += 1
            buy[0] = candle.close
            is_bought[0] = True
            coins_bought[0] = 100 / candle.close
            buy_time = candle.close_time
    elif candle['Signal'] > candle['Histogram']:
        if is_bought[0]:
            is_bought[0] = False
            margin[0] += (candle.close - buy[0]) * coins_bought[0]


    # 1
    if candle['Histogram'] > 0:
        if not is_bought[1]:
            times_bought[1] += 1
            buy[1] = candle.close
            is_bought[1] = True
            coins_bought[1] = 100 / candle.close
    elif 0 > candle['Histogram']:
        if is_bought[1]:
            is_bought[1] = False
            margin[1] += (candle.close - buy[1]) * coins_bought[1]

    # 2
    if candle['MACD'] > candle['Signal']:
        if not is_bought[2]:
            times_bought[2] += 1
            buy[2] = candle.close
            is_bought[2] = True
            coins_bought[2] = 100 / candle.close
            buy_time = candle.close_time
    elif candle['Signal'] > candle['MACD']:
            if is_bought[2]:
                is_bought[2] = False
                temp_margin = (candle.close - buy[2]) * coins_bought[2]
                margin[2] += (candle.close - buy[2]) * coins_bought[2]
                print(f"{buy_time} **BUY: {buy[2]}**, {candle.close_time} **SELL: {candle.close}**, PROFIT**** {temp_margin}")



        Week 0.0
        Margin : [0, 0, 0],
        Times Bought : [0, 0, 0],
        
2021-02-01 00:14:59.999000+01:00 **BUY: 36.3124**, 2021-02-01 01:59:59.999000+01:00 **SELL: 36.2174**, PROFIT**** -0.2616186206364737
2021-02-01 02:14:59.999000+01:00 **BUY: 36.3403**, 2021-02-01 14:14:59.999000+01:00 **SELL: 39.8318**, PROFIT**** 9.607790799745743
2021-02-01 16:14:59.999000+01:00 **BUY: 41.6636**, 2021-02-01 16:44:59.999000+01:00 **SELL: 40.35**, PROFIT**** -3.152872051382984
2021-02-01 18:59:59.999000+01:00 **BUY: 41.3412**, 2021-02-01 19:59:59.999000+01:00 **SELL: 40.8959**, PROFIT**** -1.0771337068106468
2021-02-01 22:44:59.999000+01:00 **BUY: 42.4432**, 2021-02-02 01:29:59.999000+01:00 **SELL: 42.2022**, PROFIT**** -0.5678176951784966
2021-02-02 08:59:59.999000+01:00 **BUY: 41.5207**, 2021-02-02 13:59:59.999000+01:00 **SELL: 43.0443**, PROFIT**** 3.669494974795709
2021-02-02 19:44:59.999000+01:00 **BUY: 42.0585**, 2021-02-03 02:59:59.999000+01:00 **SELL: 42.335**, PROFIT**** 0

In [95]:
prices = month

import numpy as np
prices['MACD_SIG'] = np.where(prices['MACD'] > prices['Signal'], 1, -1)
prices['RSI_SIG'] = np.where(prices['RSI'] > 30, 1, -1)

# p_margin = 0
# p_is_bought = False
# P_buy = 0
# p_times_bought = 0
# for index, x in prices.iterrows():
#     if x['MACD_SIG'] == 1 and x['RSI_SIG'] == 1:
#         if not p_is_bought:
#             p_is_bought = True
#             P_buy = x['close']
#             print(f"BOUGHT AT {P_buy}")
#             p_times_bought += 1
#     elif x['MACD_SIG'] == -1 or x['RSI_SIG'] == -1:
#         if p_is_bought:
#             p_is_bought = False
#             print(f"SOLD AT {x['close']}")
#             p_margin += x['close'] - P_buy

<ipython-input-95-bd0d579edeb9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['MACD_SIG'] = np.where(prices['MACD'] > prices['Signal'], 1, -1)
<ipython-input-95-bd0d579edeb9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['RSI_SIG'] = np.where(prices['RSI'] > 30, 1, -1)
